In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import sys
import time
from sklearn.metrics import accuracy_score
from tqdm import tqdm_notebook as tqdm

sys.path.insert(0, "../../")

from rcalgo.tf.models.semi_supervised.adversarial_bert import AdversarialBertClassificationModel, \
    AdversarialBertIdSequenceTraining
from rcalgo.tf.models.semi_supervised.data import SemiSupTrainData
from rcalgo.tf.utils.tokenizers import CharTokenizer
from rcalgo.tf.utils.serving_utils import add_prefix_to_ckpt
from rcalgo.tf.training.hook import TensorInfoPrintHook
from rcalgo.tf.training.data.data_iterator import DataIterator
from rcalgo.tf.utils.preprocessors import BertSentencePreprocessor
from rcalgo.tf.models.finetune.google_bert import BertIdSequenceClassificationModel
from rcalgo.tf.utils.serving_utils import load_checkpoint_with_name_map

In [2]:
# load data
data_dir = "/home/web_server/antispam/project/datasets/female_vulgar/"
train_df = pd.read_csv(data_dir + 'train_1w.csv')
test_df = pd.read_csv(data_dir + 'test.csv')
# count word dict
tokenizer = CharTokenizer.build_from_corpus(test_df['text'], freq_threshold=2)
word_dict = tokenizer.word2idx

In [3]:
org_unsup_file = data_dir + "unsup0.txt"

In [4]:
sup_texts = train_df['text']
labels = train_df['label']
batch_size = 32
unsup_ratio = 1
gpu_num = 2
num_epochs = 150
num_train_steps = num_epochs * (int(len(sup_texts)) // (batch_size * gpu_num))
print(f"num_train_steps: {num_train_steps}")

num_train_steps: 23400


In [5]:
# create execute model
config = {
    # uda config
    "unsup_coeff": 1.5,  # unsup loss coeff 
    "tsa_schedule": None,  # linear_schedule, exp_schedule, log_schedule, None
    "batch_size": batch_size,
    "unsup_ratio": unsup_ratio,  # unsup data ratio
    "num_train_steps": num_train_steps,  # num of train steps is needed for tas threshold calcuation
    "use_adv_train": False,
    "epsilon": 10,
    "xi": 0.1,
    "rampup_method": None, 
    
    # model config
    "max_seq_length": 32,
    "epochs": num_epochs,
    "test_size": 0.1,
    "batch_size": batch_size,
    "warmup_proportion": 0.1,
    'gpu_id': 0,
    'gpu_num': gpu_num,

    "learning_rate": 2e-5,

    "vocab_file": "/home/web_server/antispam/project/pretrained_models/"
                  "chinese_L-12_H-768_A-12/vocab.txt",
    "checkpoint_file": "/home/web_server/antispam/project/pretrained_models/"
                       "chinese_L-12_H-768_A-12/bert_model.ckpt",
    "num_classes": 2,
    "enable_nsp": False,
    "max_predictions_per_seq": 10,
    "masked_lm_probs": 0.15,
#     "hidden_dropout_prob": 0.5, 
    "freeze_layer_num": 6,
}

In [6]:
n_examples = len(train_df)
config["num_train_steps"] = int(n_examples * config["epochs"] / config["batch_size"] / gpu_num)
config["num_warmup_steps"] = int(config["num_train_steps"] * config["warmup_proportion"])
print(config)

{'unsup_coeff': 1.5, 'tsa_schedule': None, 'batch_size': 32, 'unsup_ratio': 1, 'num_train_steps': 23437, 'use_adv_train': False, 'epsilon': 10, 'xi': 0.1, 'rampup_method': None, 'max_seq_length': 32, 'epochs': 150, 'test_size': 0.1, 'warmup_proportion': 0.1, 'gpu_id': 0, 'gpu_num': 2, 'learning_rate': 2e-05, 'vocab_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/vocab.txt', 'checkpoint_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/bert_model.ckpt', 'num_classes': 2, 'enable_nsp': False, 'max_predictions_per_seq': 10, 'masked_lm_probs': 0.15, 'freeze_layer_num': 6, 'num_warmup_steps': 2343}


In [7]:
test_sup_texts = test_df['text']
test_labels = test_df['label']

In [8]:
org_unsup_texts = []
with open(org_unsup_file) as org_f:
    all_org_unsup_texts = [line.strip() for line in org_f.readlines()]

# unsup_size = len(all_org_unsup_texts) // unsup_ratio
# for i in range(unsup_size):
#     org_unsup_texts.append([all_org_unsup_texts[i * unsup_ratio + j] for j in range(unsup_ratio)])
    
test_size = len(test_labels)
test_org_unsup_texts = all_org_unsup_texts[:test_size]

In [9]:
print(f"sup_texts: ")
print(sup_texts[:5])
print(f"org_unsup_texts: ")
print(all_org_unsup_texts[:5])
print(len(all_org_unsup_texts))

sup_texts: 
0                上去不，宝贝
1            服务区这个小嘴太美了
2                 第二个雏吗
3       不要彩礼也不见得老公疼你一辈子
4    有没有发现大多都是女孩子患有抑郁症💔
Name: text, dtype: object
org_unsup_texts: 
['祝雅儿新年快乐，开心快乐每一天，越来越漂亮哈[笑哭]', '非洲哥', '鬼街在哪啊', '哈卵', '差那点洗脸洗手的时间？']
400000


In [10]:
preprocessor = BertSentencePreprocessor(config["vocab_file"])

In [11]:
train_df['input_ids'] = train_df['text'].map(
        lambda x: preprocessor.convert_text_to_ids_string(x))
test_df['input_ids'] = test_df['text'].map(
    lambda x: preprocessor.convert_text_to_ids_string(x))

## finetune with original bert

In [17]:
tf.compat.v1.reset_default_graph()
ft_model = BertIdSequenceClassificationModel(config, distribute="tf_distribute").build_model()
ft_model

2020-12-22 18:35:44,003  model.py (77) (MainThread) : INFO  config: {'unsup_coeff': 1, 'tsa_schedule': 'linear_schedule', 'batch_size': 32, 'unsup_ratio': 6, 'num_train_steps': 23437, 'use_adv_train': False, 'epsilon': 10, 'xi': 0.1, 'rampup_method': 'sigmoid', 'max_seq_length': 32, 'epochs': 150, 'test_size': 0.1, 'warmup_proportion': 0.1, 'gpu_id': 0, 'gpu_num': 2, 'learning_rate': 2e-05, 'vocab_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/vocab.txt', 'checkpoint_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/bert_model.ckpt', 'num_classes': 2, 'enable_nsp': False, 'max_predictions_per_seq': 10, 'masked_lm_probs': 0.15, 'freeze_layer_num': 6, 'num_warmup_steps': 2343}


Instructions for updating:
reduction_indices is deprecated, use axis instead



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


2020-12-22 18:35:45,961  google_bert.py (113) (MainThread) : INFO  filtered vars: [<tf.Variable 'bert/embeddings/word_embeddings:0' shape=(21128, 768) dtype=float32_ref>, <tf.Variable 'bert/embeddings/token_type_embeddings:0' shape=(2, 768) dtype=float32_ref>, <tf.Variable 'bert/embeddings/position_embeddings:0' shape=(512, 768) dtype=float32_ref>, <tf.Variable 'bert/embeddings/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>, <tf.Variable 'bert/embeddings/LayerNorm/gamma:0' shape=(768,) dtype=float32_ref>, <tf.Variable 'bert/encoder/layer_0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32_ref>, <tf.Variable 'bert/encoder/layer_0/attention/self/query/bias:0' shape=(768,) dtype=float32_ref>, <tf.Variable 'bert/encoder/layer_0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32_ref>, <tf.Variable 'bert/encoder/layer_0/attention/self/key/bias:0' shape=(768,) dtype=float32_ref>, <tf.Variable 'bert/encoder/layer_0/attention/self/value/kernel:0' shape=(768, 768) dtype=f

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [19]:
ft_model.trainer.initialize()

2020-12-22 18:36:18,832  single_worker_graph_copy_trainer.py (92) (MainThread) : INFO  Initializing trainer ...
2020-12-22 18:36:18,833  single_worker_graph_copy_trainer.py (105) (MainThread) : INFO  gpu_num: 2
2020-12-22 18:36:18,835  graph_copy.py (420) (MainThread) : INFO  build_train_graph kwargs: {'unsup_coeff': 1, 'tsa_schedule': 'linear_schedule', 'batch_size': 32, 'unsup_ratio': 6, 'num_train_steps': 23437, 'use_adv_train': False, 'epsilon': 10, 'xi': 0.1, 'rampup_method': 'sigmoid', 'max_seq_length': 32, 'epochs': 150, 'test_size': 0.1, 'warmup_proportion': 0.1, 'learning_rate': 2e-05, 'vocab_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/vocab.txt', 'checkpoint_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/bert_model.ckpt', 'num_classes': 2, 'enable_nsp': False, 'max_predictions_per_seq': 10, 'masked_lm_probs': 0.15, 'freeze_layer_num': 6, 'num_warmup_steps': 2343, 'excluded_task_set': ['pretrained_mod


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2020-12-22 18:36:27,936  graph_copy.py (444) (MainThread) : INFO  build_train_graph (bert_id_sequence_classification_model): building inputs...
2020-12-22 18:36:27,938  graph_copy.py (446) (MainThread) : INFO  task: bert_id_sequence_classification_model, <class 'str'>
2020-12-22 18:36:27,942  graph_copy.py (474) (MainThread) : INFO  build_train_graph (bert_id_sequence_classification_model): building per replica graph...



INFO:tensorflow:batch_all_reduce: 100 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10


2020-12-22 18:36:47,352  graph_copy.py (482) (MainThread) : INFO  train_ops: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Adam_bert_id_sequence_classification_model/Identity:0' shape=() dtype=int64>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'Adam_bert_id_sequence_classification_model/Identity_1:0' shape=() dtype=int64>
}], losses: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Mean:0' shape=() dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/Mean:0' shape=() dtype=float32>
}], metrics: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Cast:0' shape=(?,) dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/Cast:0' shape=(?,) dtype=float32>
}]


INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).


2020-12-22 18:36:47,411  single_worker_graph_copy_trainer.py (119) (MainThread) : INFO  all_tasks: ['bert_id_sequence_classification_model']
2020-12-22 18:36:47,825  single_worker_graph_copy_trainer.py (122) (MainThread) : INFO  init variables with sess target: None
2020-12-22 18:37:07,902  single_worker_graph_copy_trainer.py (124) (MainThread) : INFO  single_worker: initialized
2020-12-22 18:37:07,904  single_worker_graph_copy_trainer.py (125) (MainThread) : INFO  Init time cost (secs):  49.071


In [21]:
ft_model.trainer.load_single_graph_checkpoint(model_path=config['checkpoint_file'])

2020-12-22 18:37:24,277  single_worker_graph_copy_trainer.py (532) (MainThread) : INFO  Reload single graph from /home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/bert_model.ckpt
2020-12-22 18:37:24,280  single_worker_graph_copy_trainer.py (540) (MainThread) : INFO  Restore 199 vars from checkpoint.


In [24]:
ft_model.train_and_evaluate([train_df['input_ids'].values, labels], 
                            [test_df['input_ids'].values, test_labels], 
            batch_size=batch_size * config['gpu_num'],
            epochs=30,
            hooks=[])

2020-12-22 18:38:12,322  single_worker_graph_copy_trainer.py (150) (MainThread) : INFO  tasks: ['bert_id_sequence_classification_model']
2020-12-22 18:38:12,323  single_worker_graph_copy_trainer.py (151) (MainThread) : INFO  loss: Tensor("Identity:0", shape=(), dtype=float32, device=/replica:0/task:0/device:CPU:0), loss_list: [<tf.Tensor 'Identity:0' shape=() dtype=float32>], task name: bert_id_sequence_classification_model
2020-12-22 18:38:12,323  single_worker_graph_copy_trainer.py (156) (MainThread) : INFO  do_test: True
2020-12-22 18:38:12,323  single_worker_graph_copy_trainer.py (157) (MainThread) : INFO  building dataset ...
2020-12-22 18:38:12,324  data.py (19) (MainThread) : INFO  Data: will drop 16 remainder_size data items.
2020-12-22 18:38:12,401  data.py (19) (MainThread) : INFO  Data: will drop 0 remainder_size data items.
2020-12-22 18:38:13,798  single_worker_graph_copy_trainer.py (174) (MainThread) : INFO  start training sess run ...


'epoch 0 train time(sec): 23.104326725006104'
{'best_train_metrics': [0.7775],
 'best_validate_metrics': [0.8431],
 'epoch_index': 0,
 'train_loss': [0.4682],
 'train_metrics': [0.7775],
 'validate_loss': [0.3610],
 'validate_metrics': [0.8431]}


task:bert_id_sequence_classification_model, lr:2.0000e-05, loss:0.2088, metric:0.9531:   1%|          | 1/156 [00:00<00:18,  8.44it/s]  

'epoch 1 train time(sec): 16.161346197128296'
{'best_train_metrics': [0.8679],
 'best_validate_metrics': [0.8489],
 'epoch_index': 1,
 'train_loss': [0.3202],
 'train_metrics': [0.8679],
 'validate_loss': [0.3473],
 'validate_metrics': [0.8489]}


task:bert_id_sequence_classification_model, lr:2.0000e-05, loss:0.0679, metric:0.9844:   1%|          | 1/156 [00:00<00:18,  8.38it/s]  

...

task:bert_id_sequence_classification_model, lr:2.0000e-05, loss:0.0008, metric:1.0000:   1%|          | 1/156 [00:00<00:18,  8.46it/s]  

'epoch 27 train time(sec): 16.23655343055725'
{'best_train_metrics': [0.9970],
 'best_validate_metrics': [0.8733],
 'epoch_index': 27,
 'train_loss': [0.0165],
 'train_metrics': [0.9945],
 'validate_loss': [0.5860],
 'validate_metrics': [0.8726]}


task:bert_id_sequence_classification_model, lr:2.0000e-05, loss:0.0016, metric:1.0000:   1%|          | 1/156 [00:00<00:18,  8.48it/s]  

'epoch 28 train time(sec): 16.194658517837524'
{'best_train_metrics': [0.9970],
 'best_validate_metrics': [0.8733],
 'epoch_index': 28,
 'train_loss': [0.0115],
 'train_metrics': [0.9963],
 'validate_loss': [0.6255],
 'validate_metrics': [0.8647]}


'epoch 29 train time(sec): 16.212141513824463'
{'best_train_metrics': [0.9970],
 'best_validate_metrics': [0.8733],
 'epoch_index': 29,
 'train_loss': [0.0125],
 'train_metrics': [0.9959],
 'validate_loss': [0.6720],
 'validate_metrics': [0.8673]}


## finetune bert with Adversarial Training

In [12]:
unsup_text_ids = [preprocessor.convert_text_to_ids_string(text) for text in all_org_unsup_texts]
test_unsup_text_ids = [preprocessor.convert_text_to_ids_string(text) for text in test_org_unsup_texts]

In [13]:
tf.compat.v1.reset_default_graph()
exec_model = AdversarialBertClassificationModel(config, name="adv_bert_class", distribute="tf_distribute")
adv_training = AdversarialBertIdSequenceTraining(exec_model, config, name="adv_bert_train", distribute="tf_distribute")

2020-12-23 15:43:49,661  model.py (77) (MainThread) : INFO  config: {'unsup_coeff': 1.5, 'tsa_schedule': None, 'batch_size': 32, 'unsup_ratio': 1, 'num_train_steps': 23437, 'use_adv_train': False, 'epsilon': 10, 'xi': 0.1, 'rampup_method': None, 'max_seq_length': 32, 'epochs': 150, 'test_size': 0.1, 'warmup_proportion': 0.1, 'gpu_id': 0, 'gpu_num': 2, 'learning_rate': 2e-05, 'vocab_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/vocab.txt', 'checkpoint_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/bert_model.ckpt', 'num_classes': 2, 'enable_nsp': False, 'max_predictions_per_seq': 10, 'masked_lm_probs': 0.15, 'freeze_layer_num': 6, 'num_warmup_steps': 2343}


2020-12-23 15:43:49,721  model.py (77) (MainThread) : INFO  config: {'unsup_coeff': 1.5, 'tsa_schedule': None, 'batch_size': 32, 'unsup_ratio': 1, 'num_train_steps': 23437, 'use_adv_train': False, 'epsilon': 10, 'xi': 0.1, 'rampup_method': None, 'max_seq_length': 32, 'epochs': 150, 'test_size': 0.1, 'warmup_proportion': 0.1, 'gpu_id': 0, 'gpu_num': 2, 'learning_rate': 2e-05, 'vocab_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/vocab.txt', 'checkpoint_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/bert_model.ckpt', 'num_classes': 2, 'enable_nsp': False, 'max_predictions_per_seq': 10, 'masked_lm_probs': 0.15, 'freeze_layer_num': 6, 'num_warmup_steps': 2343}


In [14]:
adv_training.build_model() # 确认是否成功filter变量


Instructions for updating:
reduction_indices is deprecated, use axis instead




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updatin

2020-12-23 15:44:03,989  adversarial_bert.py (259) (MainThread) : INFO  filtered vars: [<tf.Variable 'AdvTrain/bert/embeddings/word_embeddings:0' shape=(21128, 768) dtype=float32_ref>, <tf.Variable 'AdvTrain/bert/embeddings/token_type_embeddings:0' shape=(2, 768) dtype=float32_ref>, <tf.Variable 'AdvTrain/bert/embeddings/position_embeddings:0' shape=(512, 768) dtype=float32_ref>, <tf.Variable 'AdvTrain/bert/embeddings/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>, <tf.Variable 'AdvTrain/bert/embeddings/LayerNorm/gamma:0' shape=(768,) dtype=float32_ref>, <tf.Variable 'AdvTrain/bert/encoder/layer_0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32_ref>, <tf.Variable 'AdvTrain/bert/encoder/layer_0/attention/self/query/bias:0' shape=(768,) dtype=float32_ref>, <tf.Variable 'AdvTrain/bert/encoder/layer_0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32_ref>, <tf.Variable 'AdvTrain/bert/encoder/layer_0/attention/self/key/bias:0' shape=(768,) dtype=float32_ref>, <tf.

In [15]:
adv_training.model_summary()  # 确认是否是201个变量

total parameter number: 102269186


variable_name variable_shape  \
0           AdvTrain/bert/embeddings/word_embeddings:0   [21128, 768]   
1     AdvTrain/bert/embeddings/token_type_embeddings:0       [2, 768]   
2       AdvTrain/bert/embeddings/position_embeddings:0     [512, 768]   
3            AdvTrain/bert/embeddings/LayerNorm/beta:0          [768]   
4           AdvTrain/bert/embeddings/LayerNorm/gamma:0          [768]   
..                                                 ...            ...   
196  AdvTrain/bert/encoder/layer_11/output/LayerNor...          [768]   
197                AdvTrain/bert/pooler/dense/kernel:0     [768, 768]   
198                  AdvTrain/bert/pooler/dense/bias:0          [768]   
199                     AdvTrain/bert/output_weights:0       [2, 768]   
200                        AdvTrain/bert/output_bias:0            [2]   

    parameters  
0     16226304  
1         1536  
2       393216  
3          768  
4          768  
..         ...  
196        768  
197     589824  
198        768  
199       1536  
200          2  

[201 rows x 3 columns]

In [16]:
logging_hook = TensorInfoPrintHook({"unsup_loss": adv_training.unsup_loss,
#                                     "epsilon": adv_training.epsilon,
#                                     "tsa_threshold": adv_training.tsa_threshold, 
                                    "sup_loss": adv_training.sup_loss})

In [17]:
train_data = SemiSupTrainData([train_df['input_ids'].values, unsup_text_ids, labels],
                     mode="train", batch_size=batch_size * config['gpu_num'], 
                     drop_remainder=True, name="train_data")

test_data = SemiSupTrainData([test_df['input_ids'].values, test_unsup_text_ids, test_labels],
                     mode="test", batch_size=batch_size * config['gpu_num'], 
                    drop_remainder=True, name="test_data")

2020-12-23 15:44:09,940  data.py (19) (MainThread) : INFO  train_data: will drop 16 remainder_size data items.
2020-12-23 15:44:11,040  data.py (19) (MainThread) : INFO  test_data: will drop 0 remainder_size data items.


In [18]:
adv_training.trainer.initialize()

2020-12-23 15:44:11,153  single_worker_graph_copy_trainer.py (92) (MainThread) : INFO  Initializing trainer ...
2020-12-23 15:44:11,153  single_worker_graph_copy_trainer.py (105) (MainThread) : INFO  gpu_num: 2
2020-12-23 15:44:11,155  graph_copy.py (420) (MainThread) : INFO  build_train_graph kwargs: {'unsup_coeff': 1.5, 'tsa_schedule': None, 'batch_size': 32, 'unsup_ratio': 1, 'num_train_steps': 23437, 'use_adv_train': False, 'epsilon': 10, 'xi': 0.1, 'rampup_method': None, 'max_seq_length': 32, 'epochs': 150, 'test_size': 0.1, 'warmup_proportion': 0.1, 'learning_rate': 2e-05, 'vocab_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/vocab.txt', 'checkpoint_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/bert_model.ckpt', 'num_classes': 2, 'enable_nsp': False, 'max_predictions_per_seq': 10, 'masked_lm_probs': 0.15, 'freeze_layer_num': 6, 'num_warmup_steps': 2343, 'excluded_task_set': ['pretrained_model/default'], 'p


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2020-12-23 15:44:20,831  graph_copy.py (444) (MainThread) : INFO  build_train_graph (adv_bert_train): building inputs...
2020-12-23 15:44:20,832  graph_copy.py (446) (MainThread) : INFO  task: adv_bert_train, <class 'str'>
2020-12-23 15:44:20,836  graph_copy.py (474) (MainThread) : INFO  build_train_graph (adv_bert_train): building per replica graph...



INFO:tensorflow:batch_all_reduce: 100 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10


2020-12-23 15:45:10,671  graph_copy.py (482) (MainThread) : INFO  train_ops: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Adam_adv_bert_train/Identity:0' shape=() dtype=int64>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'Adam_adv_bert_train/Identity_1:0' shape=() dtype=int64>
}], losses: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'semi-sup-loss:0' shape=() dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/semi-sup-loss:0' shape=() dtype=float32>
}], metrics: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Cast:0' shape=(?,) dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/Cast:0' shape=(?,) dtype=float32>
}]


INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).


2020-12-23 15:45:10,709  single_worker_graph_copy_trainer.py (119) (MainThread) : INFO  all_tasks: ['adv_bert_train']
2020-12-23 15:45:11,096  single_worker_graph_copy_trainer.py (122) (MainThread) : INFO  init variables with sess target: None
2020-12-23 15:45:28,580  single_worker_graph_copy_trainer.py (124) (MainThread) : INFO  single_worker: initialized
2020-12-23 15:45:28,580  single_worker_graph_copy_trainer.py (125) (MainThread) : INFO  Init time cost (secs):  77.427


In [19]:
# adv_training.trainer.load_single_graph_checkpoint(model_path=config['checkpoint_file'])

load_checkpoint_with_name_map(adv_training.trainer.graph, adv_training.trainer.session, 
                              config['checkpoint_file'], name_map_func=lambda x: 'AdvTrain/' + x, verbose=0)

Reload graph from /home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/bert_model.ckpt



2020-12-23 15:45:28,589  serving_utils.py (105) (MainThread) : INFO  Restore 199 vars from checkpoint.


In [20]:
adv_training.train_and_evaluate(train_data, test_data,
                batch_size=batch_size * config['gpu_num'],
                epochs=30,
                hooks=[logging_hook])

2020-12-23 15:45:34,070  single_worker_graph_copy_trainer.py (150) (MainThread) : INFO  tasks: ['adv_bert_train']
2020-12-23 15:45:34,071  single_worker_graph_copy_trainer.py (151) (MainThread) : INFO  loss: Tensor("Identity:0", shape=(), dtype=float32, device=/replica:0/task:0/device:CPU:0), loss_list: [<tf.Tensor 'Identity:0' shape=() dtype=float32>], task name: adv_bert_train
2020-12-23 15:45:34,071  single_worker_graph_copy_trainer.py (156) (MainThread) : INFO  do_test: True
2020-12-23 15:45:34,071  single_worker_graph_copy_trainer.py (157) (MainThread) : INFO  building dataset ...


sup_data_size: 9984, unsup_data_size: 399984


2020-12-23 15:45:36,006  single_worker_graph_copy_trainer.py (174) (MainThread) : INFO  start training sess run ...


last: {'unsup_loss': 0.019956084, 'sup_loss': 0.50901276}
'epoch 0 train time(sec): 46.03555750846863'
{'best_train_metrics': [0.7657],
 'best_validate_metrics': [0.8448],
 'epoch_index': 0,
 'train_loss': [0.5031],
 'train_metrics': [0.7657],
 'validate_loss': [0.3670],
 'validate_metrics': [0.8448]}


last: {'unsup_loss': 0.05996834, 'sup_loss': 0.20698157}


...

'epoch 27 train time(sec): 30.556280374526978'
{'best_train_metrics': [0.9974],
 'best_validate_metrics': [0.8777],
 'epoch_index': 27,
 'train_loss': [0.0717],
 'train_metrics': [0.9965],
 'validate_loss': [0.3326],
 'validate_metrics': [0.8730]}


last: {'unsup_loss': 0.046645187, 'sup_loss': 0.024455905}


  0%|          | 0/156 [00:00<?, ?it/s]

'epoch 28 train time(sec): 30.576245307922363'
{'best_train_metrics': [0.9974],
 'best_validate_metrics': [0.8777],
 'epoch_index': 28,
 'train_loss': [0.0733],
 'train_metrics': [0.9973],
 'validate_loss': [0.3310],
 'validate_metrics': [0.8746]}


last: {'unsup_loss': 0.122174956, 'sup_loss': 0.023947358}
'epoch 29 train time(sec): 30.56327509880066'
{'best_train_metrics': [0.9977],
 'best_validate_metrics': [0.8777],
 'epoch_index': 29,
 'train_loss': [0.0673],
 'train_metrics': [0.9977],
 'validate_loss': [0.3673],
 'validate_metrics': [0.8741]}


In my experiment, with 10000 training data of `female_vulgar`, the validate accuracy improved from **0.8733** to **0.8788**.